# Homework 3: LLM agents & RL fine-tuning

The third homework zooms in on the following skills: implementing an advanced generation system, diving into task-specific RL fine-tuning hands-on and critically thinking about fine-tuning of LMs.

### Logistics

* submission deadline: June 28th th 23:59 German time via Moodle
  * please upload a **SINGLE .IPYNB FILE named Surname_FirstName_HW3.ipynb** containing your solutions of the homework.
* please solve and submit the homework **individually**! 
* if you use Colab, to speed up the execution of the code on Colab, you can use the available GPU (if Colab resources allow). For that, before executing your code, navigate to Runtime > Change runtime type > GPU > Save.
* please note that we will need a lot of GPU memory for both Ex. 1 and Ex. 2 -- therefore, it might be best to do the tasks in **separate runtimes on Colab**, otherwise you might run into out of memory issues.

## Exercise 1: Building a retrieval-augmented generation system (30 points)

An increasingly popular approach to language generation is so called *retrieval-augmented generation* (RAG) wherein a language model is supplied with additional (textual) information retrieved from some storage, in addition to the actual task query. It has been found that this additional context improves model performance, and, e.g., allows to use LLMs with custom information (e.g., proprietary documents etc). 

The general set up of a RAG system is as follows:
1. Some form of a database (DB) with (searchable) relevant background information (e.g., a database, a set of documents, ...) is created.
   1. A common database format are *vector DBs*, or, vectore stores. You can optionally learn more about vector DBs, e.g., here: https://www.pinecone.io/learn/vector-database/. The important conceptual point is that some form of a searchable database with relevant (textual) information is created.
2. An LLM that will be generating the responses to the queries, given context, is chosen.
3. An embedding model is chosen.
4. Task queries (e.g., questions or instructions) are provided to the system. 
   1. The query is converted to an embedding (using the model chosen ins tep 3), and the embedding is used to search and retrieve relevant information from the database. The specific retrieval method depnds on the nature of the database.
   2. The relevant information is supplied to the LLM as context.
5. Given the extended context, the LLM provides output.

This is visualized in the figure below.

![img](../tutorials/pics/basic_rag.png)

The image is sourced from [here](https://docs.llamaindex.ai/en/stable/getting_started/concepts/).

For more details on RAG, you can read the first part of [this](https://docs.llamaindex.ai/en/stable/getting_started/concepts/) blog post (until "important concepts within each step"). [Here](https://arxiv.org/pdf/2005.11401) is an optional paper about RAG, in case you want to learn more. 

**YOUR TASK**
> Your task in this exercise is to explore RAG by implementing a RAG system for recipe generation. The implemented RAG system should be compared to the performance of the same model in a "vanilla" set-up where the model solves the task directly.
>
> We will use the package `LlamaIndex` and the LLM `phi-3-mini-4k-instruct` model as the backbone for the implementation. We will use the `BAAI/bge-small-en-v1.5` model as our embedding model.
> 
> We will use unstructured data in the form of a recipe dataset `m3hrdadfi/recipe_nlg_lite`. This dataset will be indexed and it will be used to supplement information for the LLM, additionally to the query. The train split of the dataset should be used for the index, and a sample from the test dataset will be used for sampling queries with which the system will be tested. 
>
> For this task, please complete the following steps:
> 1. Download the dataset from Huggingface. 
> 2. Briefly familiarize yourself with the dataset.
> 3. Briefly familiarize yourself with [this](https://docs.llamaindex.ai/en/stable/getting_started/starter_example_local/) LLamaIndex example RAG system.  
> 4. Complete the code below (in place of "### YOUR CODE HERE ####"), following the instructions in the comments to build a working RAG system that will generate recipes. Note that you will have to work with the LlamaIndex documentation to complete and understand the code. Some links are already provided.
> 5. Answer the questions at the end of the exercise.

In [1]:
# uncomment and run in your environment / on Colab, if you haven't installed these packages yet
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-huggingface
!pip install sentence-transformers
!pip install datasets
!pip install llama-index
!pip install "transformers[torch]" "huggingface_hub[inference]"
from IPython.display import clear_output
clear_output()

In [4]:
# import packages
from datasets import load_dataset
import os
import pandas as pd
from llama_index.core import VectorStoreIndex, Settings, Document 
# from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import AutoTokenizer
import torch

In [5]:
# load dataset from HF
dataset = load_dataset("m3hrdadfi/recipe_nlg_lite")
# convert train split to pandas dataframe
dataset_df = pd.DataFrame(dataset["train"])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/datasets/load.py:1486: FutureWarning: The repository for m3hrdadfi/recipe_nlg_lite contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/m3hrdadfi/recipe_nlg_lite
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/6118 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1080 [00:00<?, ? examples/s]

In [6]:
# explore
dataset_df.head()

,uid,name,description,link,ner,ingredients,steps
0,dab8b7d0-e0f6-4bb0-aed9-346e80dace1f,pork chop noodle soup,we all know how satisfying it is to make great...,https://www.yummly.com/private/recipe/Pork-Cho...,"bone in pork chops, salt, pepper, vegetable oi...","3.0 bone in pork chops, salt, pepper, 2.0 tabl...",season pork chops with salt and pepper . heat ...
1,b03f346bf39efcbace5d30a8f962147c8c4c361f,5 ingredient almond cake with fresh berries,this simple almond cake is made with just five...,https://www.skinnytaste.com/5-ingredient-almon...,"large eggs, large egg whites, sugar, pure vani...","3 large eggs, 3 large egg whites, 2/3 cup suga...",position a rack in the middle of the oven and ...
2,89b49e742b2c1d234b83044c14d81155dfea7f19,shrimp cakes,"these light, pan seared shrimp cakes are moist...",https://www.skinnytaste.com/shrimp-cakes/,"peeled and deveined jumbo shrimp, plus 3 table...","1 pound peeled and deveined jumbo shrimp, 1 cu...",pat shrimp dry with a paper towel and place in...
3,5db9af50-63dc-4c5b-9db1-783cf96675d3,chili roasted okra,"chili roasted okra with okra, sesame oil, red ...",https://www.yummly.com/private/recipe/Chili-Ro...,"okra, sesame oil, red pepper flakes, salt, pepper","1.0 pound okra, 1.0 tablespoon sesame oil, 1.0...",preheat the oven to 425degf . wash and dry the...
4,9b8da42d-d07c-4766-9f15-fd3fd6e19bf6,slow cooker chicken chili,warm up on a cold day with this slow cooker ch...,https://www.yummly.com/private/recipe/Slow-Coo...,"oil, chicken, chili powder, onion, jalapeno pe...","1.0 tablespoon oil, 1.0 pound chicken, 1.5 tab...",heat oil in skillet over medium high heat . ad...


In [7]:

# 1. In order to construct a VectorStorageIndex with the texts from the train dataset split, we need to 
# create list of formatted texts.
# We want to construct texts of the form: "Name of recipe \n\n ingredients \n\n steps"


texts = [ #CODE
    f"{row['name']}\n\ningredients: {row['ingredients']}\n\nsteps: {row['steps']}" 
    for _, row in dataset_df.iterrows()
]
texts[:2]

['pork chop noodle soup\n\ningredients: 3.0 bone in pork chops, salt, pepper, 2.0 tablespoon vegetable oil, 2.0 cup chicken broth, 4.0 cup vegetable broth, 1.0 red onion, 4.0 carrots, 2.0 clove garlic, 1.0 teaspoon dried thyme, 0.5 teaspoon dried basil, 1.0 cup rotini pasta, 2.0 stalk celery\n\nsteps: season pork chops with salt and pepper . heat oil in a dutch oven over medium high heat . add chops and cook for about 4 minutes, until golden brown . flip and cook 4 minutes more, until golden brown . transfer chops to a plate and set aside . pour half of chicken broth into pot, scraping all browned bits from bottom . add remaining chicken broth, vegetable broth, onion, carrots, celery and garlic . mix well and bring to a simmer . add 1 quart water, thyme, basil, 2 teaspoons salt and 1 teaspoon pepper . mix well and bring to a simmer . add chops back to pot and return to simmer . reduce heat and simmer for 90 minutes, stirring occasionally, being careful not to break up chops . transfer 

In [8]:
# 2. We construct single Documents from the texts
# these documents will be used to construct the vector database
documents = [Document(text=t) for t in texts]
documents

[Document(id_='d534558b-29e9-450d-9093-bfec2cb2d51b', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='pork chop noodle soup\n\ningredients: 3.0 bone in pork chops, salt, pepper, 2.0 tablespoon vegetable oil, 2.0 cup chicken broth, 4.0 cup vegetable broth, 1.0 red onion, 4.0 carrots, 2.0 clove garlic, 1.0 teaspoon dried thyme, 0.5 teaspoon dried basil, 1.0 cup rotini pasta, 2.0 stalk celery\n\nsteps: season pork chops with salt and pepper . heat oil in a dutch oven over medium high heat . add chops and cook for about 4 minutes, until golden brown . flip and cook 4 minutes more, until golden brown . transfer chops to a plate and set aside . pour half of chicken broth into pot, scraping all browned bits from bottom . add remaining chicken broth, vegetable broth, onion, carrots, celery and garlic . mix well and bring to a simmer . add 1 quart water, thyme, basil, 2 teaspoons salt and 1 teaspoon pepper . mix well and bring

In [9]:
# 3. We prepare some utility functions which are required for the LLM to generate maximally accurate responses 
# this includes correctly formatting the query and the context into the prompt and special tokens
# that are expected by the chosen LLM backbone.

# we format the texts into the Phi-3 prompt format
# See https://huggingface.co/microsoft/Phi-3-mini-4k-instruct
# to heck here how the prompt should look like!
def completion_to_prompt(completion):
    
    return f"Instruct: {completion}\nOutput:" # CODE


In [34]:
from transformers import BitsAndBytesConfig
!pip install flash-attention  # If it exists; otherwise, skip this line
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install accelerate bitsandbytes --upgrade

Looking in indexes: https://pypi.org/simple/


In [ ]:
!pip install flash-attn --no-build-isolation

In the next cell, the RAG building blocks are put together. Your task is to find out what the different configurations mean and correctly complete the code. 

In [ ]:
# 4. Save setting that are reused by our RAG system across queries
# you can learn more about the Settings object here: https://docs.llamaindex.ai/en/stable/module_guides/supporting_modules/settings/

# the embedding model is defined
Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5" # CODE
)

# backbone LLM is passed to the settings
# this is actually the model that is used to generate the response to the query, given retrieved info
# https://docs.llamaindex.ai/en/stable/understanding/using_llms/using_llms/ 
# and here: https://docs.llamaindex.ai/en/stable/module_guides/models/llms/usage_custom/
Settings.llm = HuggingFaceLLM(
    model_name= "microsoft/Phi-3-mini-4k-instruct", # CODE
    tokenizer_name= "microsoft/Phi-3-mini-4k-instruct", # CODE
    context_window=1024,
    max_new_tokens=128,
    generate_kwargs={"temperature": 0.7, "do_sample": True},
    completion_to_prompt=completion_to_prompt,
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16, "quantization_config": BitsAndBytesConfig(load_in_8bit=True), "trust_remote_code": True}, 
)
print("Set LLM!")

# https://docs.llamaindex.ai/en/stable/module_guides/indexing/vector_store_index/
# we create a vector store from our documents
# here, we let the VectorStore convert the documents to nodes automatically
index = VectorStoreIndex.from_documents(
    documents # CODE
)
print("Created index!")

Below is a single example for running a query with the RAG system, and inspecting various interesting aspects of the response generated by the model. Your task is, in the following, to set up a testing loop, which will test different queries with the RAG system and vanilla generation with the same LLM. Use the example as help. Provide comments explaning the single paramters for the following example, in place of "### YOUR COMMENT HERE ###".

In [ ]:
# https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/
# we define the query engine: generic interface that allows to ask questions over data
query_engine = index.as_query_engine(
    ### YOUR COMMENT HERE ###
    response_mode="compact", 
    ### YOUR COMMENT HERE ###
    similarity_top_k=3, 
    verbose=True, 
)
# https://docs.llamaindex.ai/en/stable/module_guides/querying/response_synthesizers/
response = query_engine.query("How do I make pork chop noodle soup?")
print(response)

for i, n in enumerate(response.source_nodes):
    print(f"----- Node {i} -----")
    print(n.node.get_content())
    print("score")
    print(n.score)

In [ ]:
# testing loop
rag_responses = []
vanilla_responses = []
retrieved_node_texts = []
retrieved_node_scores = []

# retrieve 20 random dish names from test dataset to test the system on
test_df = pd.DataFrame(dataset["test"]).sample(20)
test_queries = [
    f'How do I make {r["name"]}?' for
    _, r in test_df.iterrows()
]
print(test_queries[:5])

for query in test_queries[:5]:
    ### YOUR CODE HERE ###
    # run the query against the RAG system
    response_rag = 
    rag_responses.append(str(response_rag))
    
    # record the texts of the nodes that were retrieved for this query
    retrieved_node_texts.append(
        [n.node.get_content() for n in response_rag.source_nodes] #CODE
    )
    
    # record the scores of the texts of the retrieved nodes
    retrieved_node_scores.append(
        [n.score for n in response_rag.source_nodes] #CODE
    )

    # implement the "vanilla" (i.e., straightforward) generation of the response to the same query with the backbone LLM
    # Hint: check the intro-to-hf sheet for examples how to generate text with an LM
    response_vanilla = Settings.llm.generate(
        prompt=completion_to_prompt(query)
    ) # CODE
    vanilla_responses.append(response_vanilla)

In [ ]:
retrieved_node_scores
test_queries[:5]

> **Questions:**
> 
> 1. Inspect the results of the testing. (a) How often do you prefer the RAG response over the vanilla response? (b) Do you observe differences between the RAG and vanilla responses? If yes, what are these? (c) Inpsect the retrieved documents and their scores. Do they make sense for the queries? Do the scores match your intuition about their relevance for the query? 
 
> 2. What could be advantages and disadvantages of using RAG? Name 1 each. <br>
**Answer**: <br>
Advantage: Responses is more accurate and relevant <br> 
Disadvantage: require more computations and more complicated to compare

> 3. What is the difference between documents and nodes in the RAG system? <br>
**Answer**: <br>
Documents are the raw text entries from the dataset, while nodes are the processed representations of these documents used within the vector store for efficient retrieval and similarity scoring.

> 4. What does the embedding model do? What is the measure that underlies retrieval of relevant documents?  <br>
**Answer**: <br>
The embedding model converts text into dense vector representations. <br>
Usually the measure underlying is cosine similarity or Euclidean distance between these vectors.

> 5. What are different response modes of the query engine? Is the chosen mode a good choice for our application? Why (not)? <br>
Different response modes include "compact", "verbose". The "compact" mode is a good choie for our application as it generates concise recipe instructions, avoids overwhelming users with too much information.

## Exercise 2: RLHF for summarization (15 points)

In this exercise, we want to fine-tune GPT-2 to generate human-like news summaries, following a procedure that is very similar to the example of the movie review generation from [sheet 4.1](https://cogsciprag.github.io/Understanding-LLMs-course/tutorials/04a-finetuning-RL.html). The exercise is based on the paper by [Ziegler et al. (2020)](https://arxiv.org/pdf/1909.08593).

To this end, we will use the following components:
* in order to initialize the policy, we use GPT-2 that was already fine-tuned for summarization, i.e., our SFT model is [this](https://huggingface.co/Ayham/albert_gpt2_Full_summarization_cnndm)
* as our reward model, we will use a task-specific reward signal, namely, the ROUGE score that evaluates a summary generated by a model against a human "gold standard" summary.
* a dataset of CNN news texts and human-written summaries (for computing the rewards) for the fine-tuning which can be found [here](https://huggingface.co/datasets/abisee/cnn_dailymail). Please note that we will use the *validation* split because we only want to run short fine-tuning. 

**NOTE:** for building the datset and downloading the pretrained model, ~4GB of space will be used.

> **YOUR TASK:**
>
> Your job for this task is to set up the PPO-based training with the package `trl`, i.e., the set up step 3 of [this](https://cdn.openai.com/instruction-following/draft-20220126f/methods.svg) figure.
> 1. Please complete the code or insert comments what a particular line of code does below where the comments says "#### YOUR CODE / COMMENT HERE ####". For this and for answering the questions, you might need to dig a bit deeper into the working of proximal policy optimization (PPO), the algorithm that we are using for training. You can find relevant information, e.g., [here](https://huggingface.co/docs/trl/main/en/ppo_trainer).
> 2. To test your implementation, you can run the training for some steps, but you are NOT required to train the full model since it will take too long.
> 3. Answer the questions below.

In [ ]:
# !pip install trl accelerate==0.27.2 evaluate rouge_score datasets

In [2]:
# import libraries 
import torch
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

from transformers import AutoTokenizer
from datasets import load_dataset

from trl import (
    PPOTrainer,
    PPOConfig,
    AutoModelForCausalLMWithValueHead
)
import evaluate

ModuleNotFoundError: No module named 'trl'

In [ ]:
config = PPOConfig(
    model_name="gavin124/gpt2-finetuned-cnn-summarization-v2",
    learning_rate=1.41e-5,
    steps=250,
    #### YOUR COMMENT HERE (what is batch_size) ####
    batch_size=4,
    mini_batch_size=4,
    #### YOUR COMMENT HERE (what is ppo_epochs) ####
    ppo_epochs=4,
)

We load the CNN dataset into a DataFrame and and truncate the texts to 500 tokens, because we don't want the training to be too memory heavy and we want to have "open" some tokens for the generation (GPT-2's context window size is 1024). Then we tokenize each text and pad it.

In [ ]:
def build_dataset( 
        config,
        dataset_name="abisee/cnn_dailymail"
    ):
    """
    Build dataset for training. This builds the dataset from `load_dataset`.

    Args:
        dataset_name (`str`):
            The name of the dataset to be loaded.

    Returns:
        dataloader (`torch.utils.data.DataLoader`):
            The dataloader for the dataset.
    """
    tokenizer = AutoTokenizer.from_pretrained(#### YOUR CODE HERE ####)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = 'left'
    # load the datasets
    ds = load_dataset(dataset_name, '1.0.0', split="validation")

    def tokenize(sample):
        sample["input_ids"] = tokenizer.encode(
            #### YOUR CODE HERE (hint: inspect the dataset to see how to access the input text)####, 
            return_tensors="pt", 
            max_length=512, 
            truncation=True,
            padding="max_length"
        )
        # get the truncated natural text, too
        sample["query"] = tokenizer.decode(sample["input_ids"][0])
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch")
    return ds


In [ ]:
# build the dataset
dataset = build_dataset(config)

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [ ]:
# inspect a sample of the dataset
print(dataset[0])

We load the finetuned GPT2 model with a value head and the tokenizer. We load the model twice; the first model is the one that will be optimized while the second model serves as a reference to calculate the KL-divergence from the starting point.

In [ ]:
model = AutoModelForCausalLMWithValueHead.from_pretrained(#### YOUR CODE / COMMENT HERE ####)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(#### YOUR CODE / COMMENT HERE ####)
tokenizer = AutoTokenizer.from_pretrained(#### YOUR CODE / COMMENT HERE ####)

tokenizer.pad_token = tokenizer.eos_token

*AutoModelForCausalLMWithValueHead* is a model class provided by `trl` that is used for training models with RL with a *baseline*. The baseline is used as shown, e.g., on slide 76-78 of lecture 05. Specifically, the baseline is simultaneously learned during training, and learns to predict the so-called action value, namely the expected reward for generating a particular completion, given the query. This baseline is implemented as an additional (scalar output) head next to the next-token prediction head of the policy, and is called the value head. Based on the query and completion representation, it learns to predict a scalar reward which is compared to the ground truth reward from the reward model.

The PPOTrainer takes care of device placement and optimization later on:

In [ ]:
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)

In [ ]:
device = ppo_trainer.accelerator.device
if ppo_trainer.accelerator.num_processes == 1:
    device = 0 if torch.cuda.is_available() else "cpu"  # to avoid a `pipeline` bug
print("Device: ", device)

In [ ]:
rouge = evaluate.load("rouge")  

def reward_fn(
        output: list[str],
        original_summary: list[str]
    ):
    """
    #### YOUR COMMENT HERE ####
    """
    scores = []
    for o, s in list(zip(output, original_summary)):
      score = rouge.compute(predictions=[o.strip()], references=[s])["rouge1"]
      scores.append(torch.tensor(score))
      
    return scores

In [ ]:
output_max_length = 128
#### YOUR COMMENT HERE: explain what kind of decoding scheme these parameters initialize ####
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": output_max_length
}


for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    query_tensors = batch["input_ids"]
    query_tensors = [q.squeeze() for q in query_tensors]
    #### Get response from gpt2
    response_tensors = []
    for query in query_tensors:
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-output_max_length:])
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute score with the reward_fn above
    rewards = #### YOUR CODE HERE ####
    
    #### Run PPO step
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

> **QUESTIONS:**
> 
> 1. What are the three main steps in the training loop? Please name them (in descriptive words, you don't need to cite the code).
> 2. Suppose the plots below show training metrics for different runs of the summarization model training. Interpret what each of them tells us about training success; i.e., did the training go well on this run? Do we expect to get good summaries? Why? Be concise! 
> 3. We have truncated the query articles to maximally 512 tokens. Given that we are using ROUGE with respect to ground truth summaries as a reward, why might this be problematic?
> 3. [Bonus 2pts] The overall loss that is optimized during training with PPO consists of two components: the policy loss that is computed based on the completion log probability and the reward, and the value function loss which is computed based on the the predicted and received reward for a completion. These two loss components are weighed in the total loss function with the value function coefficient (`vf_coef`). Intuitively, how does it affect training if the coefficient is set to a high value? 

![img](data/rewards.png)

## Exercise 3: Aspects of fine-tuning (5 points)

> Please answer the following questions. Be concise!
>
> 1. When assistants are trained with RLHF, they are often optimized to be helpful and harmless. However, it has been observed that the goals of being harmless and helpful at the same time may be at odds. In particular, the problem of evasive behavior has been observed for models optimized for these goals. For example, [this paper](https://arxiv.org/pdf/2212.08073.pdf) mentions this problem. In your own words, please briefly describe what evasive behavior of LLMs is, give an example, and why it is a problem.
> 2. What special tokens are commonly used for chat model fine-tuning, and what is their purpose?
> 3. Please name two parameter-efficient fine-tuning techniques and briefly explain one advantage of using each technique over full-scale fine-tuning.